In [7]:
import keras

In [1]:
# fit a cnn on the fashion mnist dataset
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
# load dataset
(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
# reshape dataset to have a single channel
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))
# convert from integers to floats
trainX, testX = trainX.astype('float32'), testX.astype('float32')
# normalize to range 0-1
trainX,testX  = trainX / 255.0, testX / 255.0

# one hot encode target values
trainY, testY = to_categorical(trainY), to_categorical(testY)      #only need this if using labels

# model I found on the internet

# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
# model.add(MaxPooling2D())
# model.add(Flatten())
# model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(10, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# # fit model
# model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=2)
# # evaluate model
# loss, acc = model.evaluate(testX, testY, verbose=0)
# print(loss, acc)

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten, Reshape, Dropout, InputLayer

#this is the only one that works but terrible accuracy (only 50%)

def autoencoder_two(img_size):
    stacked_encoder = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[img_size[0], img_size[1],1]),
        keras.layers.Dense(200, activation="selu"),
        keras.layers.Dense(100, activation="selu"),
        keras.layers.Dense(30, activation="selu"),])

    stacked_decoder = keras.models.Sequential([
        keras.layers.Dense(100, activation="selu", input_shape=[30]),
        keras.layers.Dense(200, activation="selu", input_shape=[100]),
        keras.layers.Dense(img_size[0] * img_size[1], activation="sigmoid"),
        keras.layers.Reshape([img_size[0], img_size[1],1])])
    
    return stacked_encoder, stacked_decoder

In [14]:
#doesn't work because of different dimensions

def autoencoder_three(img_size):
    encoder = Sequential([
        Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', 
                     input_shape=(img_size[0], img_size[1], 1)),
        MaxPooling2D(2)
    ])
    
    decoder = Sequential([
        Flatten(),
        Dense(100, activation='relu', kernel_initializer='he_uniform'),
        Dense(10, activation='softmax')
    ])

    return encoder, decoder

In [ ]:
#the one I made and want to use but doesn't work

def build_autoencoder(img_size):
    #img size should be tuple of (height, width) of images in data set
    print(img_size)
    encoder = Sequential([
        InputLayer(input_shape=[img_size[0], img_size[1],1]),
        keras.layers.Conv2D(filters=64, kernel_size=7, activation='relu', padding='same'),
        keras.layers.MaxPooling2D(pool_size=2), 
        Dropout(0.25), 
        keras.layers.Conv2D(128, 3, activation='relu', padding='same'),  
        keras.layers.MaxPooling2D(pool_size=2),
        Dropout(0.25),
        keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
        keras.layers.MaxPooling2D(pool_size=2),
        Dropout(0.25),
        keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
        keras.layers.MaxPooling2D(pool_size=2),
        Dropout(0.4)
    ])
    
    decoder = Sequential([
        Conv2D(512, 3, activation='relu', padding='same'),
        UpSampling2D(2),
        Dropout(0.3),
        Conv2D(256, 3, activation='relu', padding='same'),
        UpSampling2D(2),
        Dropout(0.3),
        Conv2D(128, 3, activation='relu', padding='same'),
        UpSampling2D(2),
        Dropout(0.3),
        Conv2D(64, 3, activation='sigmoid', padding='same') , Reshape([img_size[0],img_size[1],1])
    ])

    return encoder, decoder



In [20]:
#compile model of choice

encoder, decoder = autoencoder_two([28,28]) 
autoencoder = Sequential([encoder, decoder])
autoencoder.compile(
    #loss="mean_squared_error", optimizer="RMSprop", metrics=['accuracy'])
    loss="binary_crossentropy", optimizer=keras.optimizers.SGD(lr=1.2), metrics=['accuracy'])

In [21]:
#train
epochs= 10
autoencoder_train = autoencoder.fit(trainX, trainX, epochs=epochs,verbose=1,
    validation_data=(trainX, trainX))

# Model evaluation
scores = autoencoder.evaluate(trainX, trainX, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

#save weights & model
# autoencoder = autoencoder.save_weights('autoencoder_weights.h5')
# autoencoder.save("autoencoder.h5py")

accuracy = autoencoder_train.history['accuracy']
val_accuracy = autoencoder_train.history['val_accuracy']
loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(len(accuracy))
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,6))
ax1.plot(epochs, accuracy, 'bo', label='Training accuracy')
ax1.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
ax1.set_title('Training and validation accuracy')
ax1.legend()
ax2.plot(epochs, loss, 'bo', label='Training loss')
ax2.plot(epochs, val_loss, 'b', label='Validation loss')
ax2.set_title('Training and validation loss')
ax2.legend()
plt.show()

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3340 - accuracy: 0.5005 - val_loss: 0.3104 - val_accuracy: 0.5044
Epoch 2/10
 901/1875 [=============>................] - ETA: 2s - loss: 0.3052 - accuracy: 0.5065

KeyboardInterrupt: 

In [ ]:
#visually inspect reconstructions to test accuracy
pred = autoencoder.predict(testX)
plt.figure(figsize=(20, 4))
print("Test Images")
for i in range(10):
    plt.subplot(2, 10, i+1)
    plt.imshow(testX[i, ..., 0], cmap='gray')
    curr_lbl = testX[i]
    #plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")
plt.show()    
plt.figure(figsize=(20, 4))
print("Reconstruction of Test Images")
for i in range(10):
    plt.subplot(2, 10, i+1)
    plt.imshow(pred[i, ..., 0], cmap='gray')  
plt.show()